## Projeto de Redes Sociais

In [2]:
import freeman as fm
import pandas as pd
import seaborn as sns
from unidecode import unidecode
import math

In [3]:
athletes = pd.read_csv('data/crossfit/2019_opens_athletes.csv')

C:\Users\lchen\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning:

Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.



In [4]:
# scores = pd.read_csv('data/crossfit/2019_opens_scores.csv')

In [5]:
athletes = athletes.dropna(subset=['affiliateid', 'affiliatename'])
men = athletes[athletes['divisionid'] == 1]
gyms = {}
for _, row in men.iterrows():
    if (row['affiliateid'] in gyms.keys()):
        if (gyms[row['affiliateid']][0] != row['affiliatename']):
            print('Duplicate', gyms[row['affiliateid']], row['affiliatename'])
    gyms[row['affiliateid']] = [row['affiliatename']]

In [6]:
for key, value in gyms.items():
    gyms[key].append(men[men['affiliateid'] == key]['overallscore'].mean())

In [7]:
# men = men.sort_values('overallscore')
# mediana = men['overallscore'].median()

In [8]:

gyms = {k: v for k, v in sorted(gyms.items(), key=lambda item: item[1][1])}

gyms_list = list(gyms.values())
# gyms_list = [0, 1, 2, 3]
if (len(gyms_list) % 2 == 0):
    median = (gyms_list[int(len(gyms_list) / 2)][1] + gyms_list[int(len(gyms_list) / 2) - 1][1]) / 2
else:
    median = gyms_list[math.floor(len(gyms_list) / 2)][1]
# print(math.trunc(len(gyms_list)/2))
print(median)

406265.5


In [20]:
tmp = 'graph [\n  directed 0\n'

for affiliateid, value in gyms.items():
    tmp += '  node [\n    id ' + str(affiliateid) + '\n    affiliate_name "' + unidecode(value[0]) + '"\n  ]\n'

i = 0
for affiliateid, value in gyms.items():
    percentage = round(i/len(gyms.items())*100, 2)
    if (percentage % 0.5 == 0):
        print('Progress: {}%'.format(percentage, ), end = '\r')
    for affiliate2id, value2 in gyms.items():
        if ((affiliateid != affiliate2id) and (((value[1] + value2[1]) / 2) > median) ):
#             tmp += '  edge [\n    source "' + str(affiliateid) +'"\n    target "' + str(affiliate2id) +'"\n  ]\n'
            pass
    i += 1
print('Progress: {}%'.format(100.0), end = '\r')
tmp += ']'
filename = 'data/gyms.gml'
with open(filename, 'w') as f:
    f.write(tmp)

# g = fm.load('data/gyms.gml')